## Neural Network

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline 

### Read Data

In [ ]:
df = pd.read_csv('../../../../datasets/parte1/dataset_cleaned.csv')

#### X and y arrays

In [ ]:
X = df.drop(['RainTomorrow'], axis=1)
y = df[['RainTomorrow']]

### Normalization

In [ ]:
scaler_X = MinMaxScaler(feature_range=(0, 1)).fit(X)
scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(y)
X = pd.DataFrame(scaler_X.transform(X[X.columns]), columns=X.columns)
y = pd.DataFrame(scaler_y.transform(y[y.columns]), columns=y.columns)

#### Train Test Split

Now let's split the data into a training set and a testing set. We will train out model on the training set and then use the test set to evaluate the model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023)

### Build Model

In [ ]:
def build_model(activation = 'relu', learning_rate = 0.001):
    model = Sequential()
    model.add(Dense(16, input_dim = X.shape[1], activation = activation))
    model.add(Dense(8, activation = activation))
    model.add(Dense(1, activation = 'sigmoid')) # output 

    #Compile the model 
    model.compile(
        loss = 'binary_crossentropy', 
        optimizer  = tf.optimizers.Adam(learning_rate),
        metrics = ['accuracy']) 
    return model 
    

In [ ]:
model = build_model()
model.summary()

#### Training 

Using GridSearchCV to find the best hyperparameters

In [ ]:

param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad'] 
} 

In [ ]:
kf = KFold(n_splits = 5, shuffle= True, random_state=2023)

In [ ]:
model = KerasClassifier(model=build_model, batch_size=32, validation_split=0.2, epochs = 20)

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring='accuracy', refit=True, verbose=2, n_jobs=-1)

#### Fit Model

In [ ]:
grid_search.fit(X_train, y_train)

Inspect the best parameters

In [ ]:
best_mlp_model = grid_search.best_estimator_
print(best_mlp_model)

#### Fit Model

In [ ]:
best_mlp_model.fit(X_train, y_train, epochs = 20, validation_data = (X_test, y_test), verbose = 1)

#### Learning Curves

In [ ]:
plt.plot(best_mlp_model.history_['loss'])
plt.plot(best_mlp_model.history_['val_loss'])
plt.title('Model Performance')
plt.ylabel('Loss values')
plt.xlabel('Epochs')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
predictions = best_mlp_model.predict(X_test)

#### Classification Report

In [ ]:
print(classification_report(y_test,predictions))

#### Confusion matrix

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, predictions)
plt.show()